<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-3)" data-toc-modified-id="Общее-впечатление-(ревью-3)-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 3)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

In [2]:
df= pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
display(df.head())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
df.describe()

,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [6]:
df.isna().sum() / len(df)

text     0.0
toxic    0.0
dtype: float64

Данные чисты!

In [7]:
#Проверим соотношение токсичных комментариев
df['toxic'].mean()

0.10167887648758234

Токсичных комментариев примерно 10%

In [8]:
#Лемматизируем данные
m = Mystem() 
corpus = df['text'].values.astype('U')

nltk.download('averaged_perceptron_tagger')



def lemmatize(string):
    lemmatizer = WordNetLemmatizer()
    lem = [lemmatizer.lemmatize(w) for w in nltk.word_tokenize(string)]
    result = " ".join(lem)
    return result

def clean(text):
    
    text = text.lower()    
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    
    return text
df['text_clean'] = df['text'].apply(clean)
df['text_lemma'] = df['text_clean'].apply(lemmatize)
display(df.head())


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text,toxic,text_clean,text_lemma
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,daww he matches this background colour im seem...,daww he match this background colour im seemin...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man im really not trying to edit war its j...,hey man im really not trying to edit war it ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i cant make any real suggestions on impro...,more i cant make any real suggestion on improv...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...,you sir are my hero any chance you remember wh...


In [9]:
#сбалансируем данные
#df_toxic = df[df['toxic'] == 1]
#df_not_toxic = df[df['toxic'] == 0]

#df_downsampled = pd.concat([df_toxic] + [df_not_toxic.sample(n=16225, random_state=)])
#df_downsampled['toxic'].value_counts()

train, test = train_test_split(df, test_size=0.2, shuffle=True, random_state=12345)
#train=train.sort_index(inplace=True)
display(test.shape[0])

31915

In [10]:
#features = df_downsampled['text_lemmas']
#target = df_downsampled['toxic']

#features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, shuffle=True, random_state=12345)

features_test = test['text_lemma']
target_test = test['toxic']

#сбалансируем обучающую выборку
train_toxic = train[train['toxic'] == 1]
train_not_toxic = train[train['toxic'] == 0]

train_downsampled = pd.concat([train_toxic] + [train_not_toxic.sample(n=30000, random_state=12345)])
train_downsampled['toxic'].value_counts()

train_downsampled = train_downsampled.dropna()
features_train = train_downsampled['text_lemma']
target_train = train_downsampled['toxic']
print(train_downsampled['toxic'].shape[0])


42986


## Обучение

In [12]:
#вычислим TF-IDF
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)  
tf_idf = count_tf_idf.fit_transform(features_train)


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# Посмотрим на параметры логистической регрессии

parameters = {'C': (0.1, 0.5, 1.0), 'fit_intercept': (True, False), 'solver': ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')}
model = LogisticRegression(random_state=12345, class_weight='balanced')
grid = GridSearchCV(model, parameters, cv=3, scoring=make_scorer(f1_score))
grid.fit(tf_idf, target_train)

grid.best_params_


In [14]:
#LogisticRegression
model = LogisticRegression(random_state=12345, class_weight='balanced', fit_intercept=True, C=1.0, solver='newton-cg')
model.fit(tf_idf, target_train)

tf_idf_test = count_tf_idf.transform(features_test)
predicted = model.predict(tf_idf_test)

f1_score(target_test, predicted)

0.71

In [15]:
# посмотрим на параметры SVC
parameters = {'C': (0.5, 1.0), 'kernel': ('linear', 'poly', 'rbf')}
model = SVC(random_state=12345, class_weight='balanced')
grid = GridSearchCV(model, parameters, cv=3,scoring=make_scorer(f1_score))
grid.fit(tf_idf, target_train)

grid.best_params_

In [16]:
#SVC
model = SVC(random_state=12345, class_weight='balanced', C=1.0, kernel='linear', gamma='auto')
model.fit(tf_idf, target_train)
predicted = model.predict(tf_idf_test)

f1_score(target_test, predicted)

` 0.72

In [ ]:
#LGBMClassifier
tfidf_vec = TfidfVectorizer(dtype=np.float32, sublinear_tf=True, use_idf=True, smooth_idf=True)
features_train_tfidf = tfidf_vec.fit_transform(features_train)


clf_LGBM = LGBMClassifier(verbose=-1, learning_rate=0.5, max_depth=50, num_leaves=50, n_estimators=150, max_bin=2000)
clf_LGBM.fit(tf_idf, target_train, verbose=-1)
predicted_LGBM = clf_LGBM.predict(tf_idf_test)
f1_score(target_test, predicted_LGBM)

## Выводы

Таким образом у меня получились следующие значения f1 метрики :
для LogisticRegression 0.71, 
для SVC                0.72, 
для LGBMClassifier     0.751.
У меня постоянно вылетало ядро я не смогла добиться более лучшего результата

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны